In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import os
import glob

from matplotlib.pyplot import cm
%matplotlib inline

from motion_model import sample_motion_model_velocity, predict_trajectory, plot_trajectory, plot_real_trajectories

In [ ]:

straight = np.genfromtxt("straight/straight.csv", delimiter=',', skip_header=1)
left = np.genfromtxt("left/left.csv", delimiter=',', skip_header=1)
slight_left = np.genfromtxt("slight_left/slight_left.csv", delimiter=',', skip_header=1)
right = np.genfromtxt("right/right.csv", delimiter=',', skip_header=1)
slight_right = np.genfromtxt("slight_right/slight_right.csv", delimiter=',', skip_header=1)


In [ ]:
def plot_poses(X, Y, theta, color='Teal'):
    UN1 = np.cos(theta)
    VN1 = np.sin(theta)

    plt.quiver(X, Y, UN1, VN1, color=color, headlength=7)
    pass


def plot_motion_model(trajectory_dir, sampled_paths, axes_limits, figure_size, movement_name):
    plt.figure(figsize=figure_size)
    plot_trajectory(sampled_paths)
    plt.ylabel("Y coordinate (mm)")
    plt.xlabel("X coordinate (mm)")
    plt.title("Sampled trajectories from motion model (%s)" % (movement_name))
    plt.axis('equal')
    plt.axis(axes_limits)
    plt.grid()
    plt.show()

    end_poses = None
    plt.figure(figsize=figure_size)
    for f in glob.glob(trajectory_dir + "*.csv"):
        data = np.genfromtxt(f, delimiter=',')
        delta_theta = data[0, 3]
        R = np.array([[np.cos(delta_theta), -np.sin(delta_theta)],
                      [np.sin(delta_theta),  np.cos(delta_theta)]])
        poses = np.zeros((1, 3))
        poses = data[:, 1:] - data[0, 1:]

        poses[:, :2] = np.dot(poses[:, :2], R)
        plt.plot(poses[:, 0], poses[:, 1])
        end_poses = poses[-1:, :] if end_poses is None else np.append(end_poses, poses[-1:, :], axis=0)
        pass

    plt.ylabel("Y coordinate (mm)")
    plt.xlabel("X coordinate (mm)")
    plt.title("Recorded trajectories from AICISS lab (%s)" % (movement_name))
    plt.axis('equal')
    plt.axis(axes_limits)
    plt.grid()
    plt.show()
    
    plot = plt.figure(figsize=(10,10))
    red_patch = mpatches.Patch(color='red', label='Recorded data')
    teal_patch = mpatches.Patch(color='Teal', label='Sampled data')
    plt.legend(handles=[red_patch, teal_patch], bbox_to_anchor=(1.05, 1), loc=2)
    plt.ylabel('y (mm)')
    plt.xlabel('x (mm)')
    plt.title('Plot of robot end poses for %s' % (movement_name))
    plot_poses(end_poses[:, 0], end_poses[:, 1], end_poses[:, 2], color='red')
    plot_poses(sampled_paths[:, -1, 0], sampled_paths[:, -1, 1], sampled_paths[:, -1, 2], color='Teal')
    plt.grid()
    axes = plt.gca()
    plt.show()

    return


In [ ]:
alpha = [  1.14270744e-02,   9.62201025e-06,   3.81152799e-05,   8.53983937e-06,
   3.28511879e-05,   1.10914099e-06]
u = [100, 0]
pose_start = [0, 0, 0]
delta_t = 0.1
duration = 6
repeat = 20
paths = predict_trajectory(u, pose_start, alpha, delta_t, duration, repeat, single_gaussian=True)

plot_motion_model("aiciss/straight/", paths, [0, 700, -100, 100], (20, 10), "straight movement")

In [ ]:
u = [100, np.radians(4.77)]

paths = predict_trajectory(u, pose_start, alpha, delta_t, duration, repeat, single_gaussian=True)

plot_motion_model("aiciss/slight_left/", paths, [0, 900, -50, 250], (20, 10), "slight left movement")


In [ ]:
u = [100, np.radians(-4.77)]

paths = predict_trajectory(u, pose_start, alpha, delta_t, duration, repeat, single_gaussian=True)

plot_motion_model("aiciss/slight_right/", paths, [0, 700, -250, 50], (20, 10), "slight right movement")


In [ ]:
u = [100, np.radians(-22.92)]

paths = predict_trajectory(u, pose_start, alpha, delta_t, duration, repeat, single_gaussian=True)

plot_motion_model("aiciss/right/", paths, [-400, 550, -500, 50], (10, 10), "right movement")

In [ ]:
u = [100, np.radians(22.92)]

paths = predict_trajectory(u, pose_start, alpha, delta_t, duration, repeat, single_gaussian=True)

plot_motion_model("aiciss/left/", paths, [0, 550, -50, 500], (10, 10), "left movement")

In [ ]:
def sample_normal_distribution(variance):
    rand_sum = 0
    for i in range(12):
        rand_sum = rand_sum + random.uniform(-1, 1)
    return rand_sum * variance / 6


In [ ]:
v = 100
omega = 0
sample_straight = np.zeros((30, 3))
# straight samples
for i in range(30):
    sample_straight[i] = sample_motion_model_velocity([v, omega], [0, 0, 0], alpha, straight[1][0])
    pass

In [ ]:
files = list()
for f in glob.glob("aiciss/straight/*.csv"):
    files.append(f)
    pass
# print("\n".join(files))


In [ ]:
plot = plt.figure(figsize=(10,10))
plt.title('Plot of robot poses after straight forward commands')
plot_poses(sample_straight[:, 0], sample_straight[:, 1], sample_straight[:, 2],)
plot_poses(straight[:, 1], straight[:, 2], straight[:, 3], color='Red')

axes = plt.gca()
axes.set_xlim([520,650])

plt.grid()
plt.show(plot)

In [ ]:
v = 100
omega = np.radians(22.92)
sample_left = np.zeros((30, 3))
for i in range(30):
    sample_left[i] = sample_motion_model_velocity([v, omega], [0, 0, 0], alpha, left[1][0])
    pass

plot = plt.figure(figsize=(10,10))
plt.title('Plot of robot poses after left commands')
plot_poses(sample_left[:, 0], sample_left[:, 1], sample_left[:, 2])
plot_poses(left[:, 1], left[:, 2], left[:, 3], color='Red')

plt.grid()
plt.show(plot)

In [ ]:
v = 100
omega = np.radians(4.77)
sample_slight_left = np.zeros((30, 3))
for i in range(30):
    sample_slight_left[i] = sample_motion_model_velocity([v, omega], [0, 0, 0], alpha, slight_left[1][0])
    pass

plot = plt.figure(figsize=(10,10))
plt.title('Plot of robot poses after slight left commands')
plot_poses(sample_slight_left[:, 0], sample_slight_left[:, 1], sample_slight_left[:, 2])
plot_poses(slight_left[:, 1], slight_left[:, 2], slight_left[:, 3], color='Red')

axes = plt.gca()
axes.set_xlim([500,625])
red_patch = mpatches.Patch(color='red', label='Original data')
teal_patch = mpatches.Patch(color='Teal', label='Sampled data')
plt.legend(handles=[red_patch, teal_patch], bbox_to_anchor=(1.05, 1), loc=2)
plt.grid()
plt.show(plot)

In [ ]:
v = 100
omega = np.radians(-22.92)
sample_right = np.zeros((30, 3))
for i in range(30):
    sample_right[i] = sample_motion_model_velocity([v, omega], [0, 0, 0], alpha, right[1][0])
    pass

plot = plt.figure(figsize=(10,10))
plt.title('Plot of robot poses after right commands')
plot_poses(sample_right[:, 0], sample_right[:, 1], sample_right[:, 2])
plot_poses(right[:, 1], right[:, 2], right[:, 3], color='Red')

axes = plt.gca()
axes.set_ylim([-450,-200])
red_patch = mpatches.Patch(color='red', label='Original data')
teal_patch = mpatches.Patch(color='Teal', label='Sampled data')
plt.legend(handles=[red_patch, teal_patch], bbox_to_anchor=(1.05, 1), loc=2)
plt.grid()
axes = plt.gca()
plt.show(plot)

In [ ]:
v = 100
omega = np.radians(-4.77)
sample_slight_right = np.zeros((30, 3))
for i in range(30):
    sample_slight_right[i] = sample_motion_model_velocity([v, omega], [0, 0, 0], alpha, slight_right[1][0])
    pass

plot = plt.figure(figsize=(10,10))
plt.title('Plot of robot poses after slight right commands')
plot_poses(sample_slight_right[:, 0], sample_slight_right[:, 1], sample_slight_right[:, 2])
plot_poses(slight_right[:, 1], slight_right[:, 2], slight_right[:, 3], color='Red')

axes = plt.gca()
axes.set_xlim([500,625])
red_patch = mpatches.Patch(color='red', label='Original data')
teal_patch = mpatches.Patch(color='Teal', label='Sampled data')
plt.legend(handles=[red_patch, teal_patch], bbox_to_anchor=(1.05, 1), loc=2)
plt.grid()
plt.show(plot)